In [5]:
import pandas as pd
from utils import load_filtered_data

In [6]:
city = 'Sicklerville'
name = 'users&bgt'

# load city data
data = load_filtered_data(city)

In [7]:
# creating the nodes

nodes = data['user']

# rename user_id column to id
# rename name (column name that cant be used because of igraph) to people_name
nodes = nodes.rename(columns={'user_id': 'id', 'name': 'people_name'})

nodes

,id,people_name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,RgDVC3ZUBqpEe6Y1kPhIpw,Monica,1282,2009-02-23 14:08:31,12640,10005,11149,"2009,2010,2011,2012,2013,2014,2015,2016","sHozd2pcOKwHtPr8VlZJfg, 7mL9cvICl8fuCQTM89a-SA...",804,...,102,108,138,65,898,2584,1731,1731,551,177
1,zkamNMEjihh3zN7lC7_WVw,Matthew,15,2006-02-04 15:53:46,35,4,22,NaN,"YLzb3jjc3p-XAOBRggcrgA, 9lcq01JUhRY2yW8AxU3TIA...",1,...,1,0,0,0,1,3,2,2,0,0
2,gVFxZMcuG_Tal2_TnpmUPg,Stephanie,146,2008-05-05 00:16:33,347,100,124,"2011,2012,2013,2014,2015,2016","uUh_q6rVMhKxwt0eaiphRg, jRqWx2z7jdSzpvqqswqNnw...",6,...,13,1,0,0,10,8,21,21,8,0
3,g0_x4kVvJAYuk96oCcbOmw,Michael,455,2009-04-15 12:46:06,1823,696,663,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2018","_BcWyKQL16ndpBdggh2kNA, mx50KP6rXgutH36fhnL6Vw...",46,...,11,0,1,5,39,33,57,57,32,8
4,-NbeVN5tnwdyYAvdNkKMjw,Dominic,1101,2012-04-25 19:31:00,6704,1959,4179,"2012,2013,2014,2015,2016,2017,2018,2019,20,20,...","9IRuYmy5YmhtNQ6ei1p-uQ, 6NvkQATK0J2nmEWsuzhHig...",193,...,33,18,2,2,312,457,637,637,137,63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3073,CUE13op55l9WZp6fBTUWnA,Nicole,2,2016-08-19 13:24:47,0,0,0,NaN,None,0,...,0,0,0,0,0,0,0,0,0,0
3074,cACn6Qc1owO7hV_ZhcP74Q,Allison,1,2019-11-10 00:05:23,0,0,0,NaN,None,0,...,0,0,0,0,0,0,0,0,0,0
3075,SpTYEhfInvALG1le1uu21w,Ray,2,2016-01-18 16:55:04,5,0,0,NaN,None,0,...,0,0,0,0,0,0,0,0,0,0
3076,sGOCaCP7_SMlaRKPNO__CQ,Tahjae,2,2019-05-04 13:07:23,0,0,0,NaN,None,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# creating the edges

temp_edges_struct = {'from': [], 'to': [], 'weight': [], 'type': []}
connected = {}

for business in data['review']['business_id'].unique():
    reviews_of_business = data['review'][data['review']['business_id'] == business]
    users_that_reviewed_business = reviews_of_business['user_id'].unique()
    for i in range(len(users_that_reviewed_business)-1):
        for j in range(i+1, len(users_that_reviewed_business)):
            if users_that_reviewed_business[i] in connected:
                connected[users_that_reviewed_business[i]].append(users_that_reviewed_business[j])
            else:
                connected[users_that_reviewed_business[i]] = [users_that_reviewed_business[j]]
            if users_that_reviewed_business[j] in connected:
                connected[users_that_reviewed_business[j]].append(users_that_reviewed_business[i])
            else:
                connected[users_that_reviewed_business[j]] = [users_that_reviewed_business[i]]

# now do it for tip (data['tip'] has user_id and business_id)
for business in data['tip']['business_id'].unique():
    tip_of_business = data['tip'][data['tip']['business_id'] == business]
    users_that_tipped_business = tip_of_business['user_id'].unique()
    for i in range(len(users_that_tipped_business)-1):
        for j in range(i+1, len(users_that_tipped_business)):
            if users_that_tipped_business[i] in connected:
                connected[users_that_tipped_business[i]].append(users_that_tipped_business[j])
            else:
                connected[users_that_tipped_business[i]] = [users_that_tipped_business[j]]
            if users_that_tipped_business[j] in connected:
                connected[users_that_tipped_business[j]].append(users_that_tipped_business[i])
            else:
                connected[users_that_tipped_business[j]] = [users_that_tipped_business[i]]


for business in data['review']['business_id'].unique():
    reviews_of_business = data['review'][data['review']['business_id'] == business]
    users_that_reviewed_business = reviews_of_business['user_id'].unique()
    for i in range(len(users_that_reviewed_business)-1):
        for j in range(i+1, len(users_that_reviewed_business)):
            # from i to j
            temp_edges_struct['from'].append(users_that_reviewed_business[i])
            temp_edges_struct['to'].append(users_that_reviewed_business[j])
            temp_edges_struct['weight'].append(connected[users_that_reviewed_business[i]].count(users_that_reviewed_business[j]))
            temp_edges_struct['type'].append('reviewed-same-restaurant')
            # from j to i
            temp_edges_struct['from'].append(users_that_reviewed_business[j])
            temp_edges_struct['to'].append(users_that_reviewed_business[i])
            temp_edges_struct['weight'].append(connected[users_that_reviewed_business[j]].count(users_that_reviewed_business[i]))
            temp_edges_struct['type'].append('reviewed-same-restaurant')

for business in data['tip']['business_id'].unique():
    tip_of_business = data['tip'][data['tip']['business_id'] == business]
    users_that_tipped_business = tip_of_business['user_id'].unique()
    for i in range(len(users_that_tipped_business)-1):
        for j in range(i+1, len(users_that_tipped_business)):
            # from i to j
            temp_edges_struct['from'].append(users_that_tipped_business[i])
            temp_edges_struct['to'].append(users_that_tipped_business[j])
            temp_edges_struct['weight'].append(connected[users_that_tipped_business[i]].count(users_that_tipped_business[j]))
            temp_edges_struct['type'].append('tipped-same-restaurant')
            # from j to i
            temp_edges_struct['from'].append(users_that_tipped_business[j])
            temp_edges_struct['to'].append(users_that_tipped_business[i])
            temp_edges_struct['weight'].append(connected[users_that_tipped_business[j]].count(users_that_tipped_business[i]))
            temp_edges_struct['type'].append('tipped-same-restaurant')

edges = pd.DataFrame(temp_edges_struct)

# remove repeated lines
edges = edges.drop_duplicates()

# now sum the weights of the repeated
edges = edges.groupby(['from', 'to', 'type']).agg({'weight': 'sum'}).reset_index()


edges

,from,to,type,weight
0,-0MIp6WKJ8QvGnYZQ5ETyg,-ZCf13dTuzGKMwKFRxTUwQ,reviewed-same-restaurant,1
1,-0MIp6WKJ8QvGnYZQ5ETyg,-u5CsCUHOvP5OHYaNGteZw,reviewed-same-restaurant,1
2,-0MIp6WKJ8QvGnYZQ5ETyg,0cLvdT2ANqM7CL93gccVjA,reviewed-same-restaurant,1
3,-0MIp6WKJ8QvGnYZQ5ETyg,2K6e4rSuEYKJpmU37Kk-oA,reviewed-same-restaurant,1
4,-0MIp6WKJ8QvGnYZQ5ETyg,2RCnyrzyOQ1oA9jSgL7XPQ,reviewed-same-restaurant,1
...,...,...,...,...
279793,zyTCJLh4oUsuRar4kxRfgg,vnSNGuRxa9p3Xrvlj3t03g,reviewed-same-restaurant,1
279794,zyTCJLh4oUsuRar4kxRfgg,w7QCKFrjpaBHiPSOo2O3MQ,reviewed-same-restaurant,1
279795,zyTCJLh4oUsuRar4kxRfgg,xvdkeWY0421VZicMbYmz2w,reviewed-same-restaurant,1
279796,zyTCJLh4oUsuRar4kxRfgg,y2PUG5elRIz6xkQP30DScA,reviewed-same-restaurant,1


In [9]:
# save data
nodes.to_csv(f'nodes_and_edges/{city}_{name}_nodes.csv', index=False)
edges.to_csv(f'nodes_and_edges/{city}_{name}_edges.csv', index=False)